**Required output_attr_stats.csv, 1 cra city commisioner.xlsx, and emails-combine 06162020.xlsx**   
**Generates unique_people.txt, commissioner_dict.json, cra_dict.json, gorvenment_people_list.txt** 
**Generates commissioner_idx_list.txt, cra_idx_list.txt, government_idx_list.txt** 



In [97]:
import csv
import pandas as pd
from tqdm import tqdm_notebook, tnrange
from datetime import datetime
import numpy as np
import json

# read file

In [98]:
df = pd.read_csv('output_attr_stats.csv')
length = len(df)

In [99]:
org=pd.read_excel('Organizations.xlsx')

In [100]:
org_keep=org[org['Keep']==1]
org_keep

,Domain by number of email addresses,Keep,combine,Name
1,talgov.com,1,NaN,City of Tallahassee
7,fsu.edu,1,2.0,Florida State U
8,leoncountyfl.gov,1,NaN,leon County
9,famu.edu,1,NaN,Florida A&M U
11,leonschools.net,1,3.0,leon County Schools
12,doh.state.fl.us,1,NaN,Florida Department of Health
13,tallahassee.com,1,NaN,Tallahassee Democrat
15,my.fsu.edu,1,2.0,Florida State U
17,tcc.fl.edu,1,NaN,Tallahassee Community College
19,admin.fsu.edu,1,2.0,Florida State U


In [101]:
org_keep_domain=org_keep['Domain by number of email addresses'].tolist()

# collect organization people

## collect all unique people

In [102]:
def get_people_list(from_list, to_list, cc_list):
    '''
    Collect all unique people
    
    from_list: list of senders
    to_list: list of recipients, ';' seperated
    cc_list: list of cc recipients, ';' seperated
    return: unique_people: a people list in alphabetical order
    '''
    unique_people = set()
    for i in range(len(from_list)):
        if not pd.isnull(from_list[i]):
            unique_people.add(from_list[i])

    for i in range(len(to_list)):
        if not pd.isnull(to_list[i]):
            for lst in to_list[i].split(';'):
                unique_people.add(lst)
                
    for i in range(len(cc_list)):
        if not pd.isnull(cc_list[i]):
            for lst in cc_list[i].split(';'):
                unique_people.add(lst)

    unique_people = sorted(list(unique_people))
    return unique_people

In [103]:
from_list = df['From'].values.tolist()
to_list = df['To'].values.tolist()
cc_list = df['CC'].values.tolist()
unique_people = get_people_list(from_list, to_list, cc_list)

In [104]:
with open('unique_people.txt', 'w', encoding='utf-8') as f:
    for line in unique_people:
        f.write(f"{line}\n")

## collect organization people

In [105]:
org_people=[]

for indx in range(len(org_keep_domain)):
    k=org_keep_domain[indx]
    for select in unique_people:
        if  k in select:
             org_people.append(select)


In [109]:
#delete the repeat one


org_people_no_repeate=list(set(org_people))

In [110]:
with open('organization_people.txt', 'w', encoding='utf-8') as f:
    for line in org_people_no_repeate:
        f.write(f"{line}\n")

## build up dictionary between organization domain and email

In [111]:
org2email={}

for indx in range(len(org_keep_domain)):
    k=org_keep_domain[indx]

    for select in unique_people:
        if  k in select:
            if k in org2email:
                org2email[k].append(select)
            else:
                org2email[k] = [select]


In [112]:
org2email['talgov.com']

['.williams@talgov.com',
 '3519@cityprojects.talgov.com',
 '4335@cityprojects.talgov.com',
 '850)545-2095edward.kring@talgov.comsentfrommyiphonethanks,eddieedwardmarwkringci-zenengagementambassador,departmentofcommunica-onsoﬃce:(850)891-8081cell:(850',
 '@talgov.com',
 'adam.jacobs@talgov.com',
 'adam.miller@talgov.com',
 'administrator@talgov.com',
 'adrienne.bellflower@talgov.com',
 'advisoryboards@talgov.com',
 'aeshah.mcqueen@talgov.com',
 'aides@talgov.com',
 'aiwu.yao@talgov.com',
 'akhenaton.thomas@talgov.com',
 'alan.williams@talgov.com',
 'alex.arbona@talgov.com',
 'alexa.hehl@talgov.com',
 'alexanderr@talgov.com',
 'ali.fleming@talgov.com',
 'alison.faris@talgov.com',
 'alissa.meyers@talgov.com',
 'allen.secreast@talgov.com',
 'allen.secrest@talgov.com',
 'allen.thompson@talgov.com',
 'allie.fleming@talgov.com',
 'allie.fleming@talgov.com>',
 'allie.flemming@talgov.com',
 'allison.behrman@talgov.com',
 'allison.fleming@talgov.com',
 'allison.fleming@talgov.com>',
 'allison_be

In [89]:
#org_keep.groupby('Name')[['Name','Domain by number of email addresses']].apply(list).values

name2domains = {}
for _,x in org_keep.groupby('Name'):
    names = x['Name'].unique()[0].replace(u'\xa0', u'')
    domains = x['Domain by number of email addresses'].values.tolist()
    
    
    if names in name2domains:
        print("Error,duplicate ", names)
    else:
        name2domains[names] = []
        
    for dom in domains:
        if dom in org2email.keys():
            name2domains[names]=name2domains[names] + org2email[dom]
        else:
            print("failed to find ", dom)
            
    
#list(df.groupby('A')['B'].apply(list).values)
#[x['B'].values.tolist() for _,x in df.groupby('A')]

In [113]:
#create a dictionary to drop the duplicate value
name2domains_no_duplicate_value = {}

for key in name2domains.keys():
    name2domains_no_duplicate_value[key]=list(set(name2domains[key]))
    





In [114]:
with open('top_organization.json', 'w') as f:
    json.dump(name2domains_no_duplicate_value, f)


## get the index of organization people

In [116]:
with open('unique_people.txt', encoding = 'utf-8') as f:
    unique_people = f.read().splitlines()

In [118]:
org_idx_list = []
for i,people in enumerate(unique_people):
    if people in org_people_no_repeate:
        org_idx_list.append(i)


In [120]:
with open('org_idx_list.txt', 'w', encoding='utf-8') as f:
    for line in org_idx_list:
        f.write(f"{line}\n")